In [29]:
import os
import sys
import re
from selenium import webdriver

os.path.dirname(sys.executable)
webDriver = webdriver.Firefox()
webDriver.get("https://hkpn.gst.dk/mapviewer.aspx?type=lkMaalebordsblade&id=10828")
menu = webDriver.find_element_by_id("metadatabutton")
m = webDriver.find_element_by_xpath("//*[text()='Kortblad']")
data= webDriver.find_element_by_id("kortbladmdBoxDiv")
menu.click()
m.click()
print(data.text)
webDriver.close()

Metadata for kortblad
Id: 10828
LandkortBladId: 10828
LandkortTypeId: 6
Maalestok: 20000
Kortblad: 3418
Version: 1966
Navn: Romsø
OpmaalingAfsluttet: 1863
Rettet: 1943
EnkelteRettelser: 1960
Udgivet: 1966
Indgang: M/20L 3418
Projektion: GS
Note4: 2 meters ækvidistance
BBox: null


In [ ]:
# Other project

import requests as req
import pandas as pd
import numpy as np

class processRec:
    
    def __init__(self,inputDataPath="garbage/UpworkProjects/Geocoding/input.csv",outputDataPath="garbage/UpworkProjects/Geocoding/output1.csv"):
        self.OWNER = "owner"
        self.ACRES = "acres_owned"
        self.ADDRESS = "address"
        self.LAT = "Latitude"
        self.LONG = "Longitude"
        self.OPLAT = "Address Latitude"
        self.OPLONG = "Address Longitude"
        self.OPACRES = "acres_owned"
        self.PARCELCOUNT = "parcel_count"
        self.OPCLOSEST = "closest parcel (miles)"
        self.OPFARTHEST = "farthest parcel (miles)"
        self.columns = [OWNER,ADDRESS,OPACRES,PARCELCOUNT,OPLAT,OPLONG,OPCLOSEST,OPFARTHEST]
        self.inputcsvPath=inputDataPath
        self.outputDataPath = outputDataPath
        self.coordDict = {}
        self.encodedAddress = []
        self.CoordFromAdd = []
        
    def getUniqueData(self,save=True):
        inputData = pd.read_csv(self.inputcsvPath)
        uo = inputData.loc[:,OWNER].unique()
        oa = inputData.loc[:,["owner","address"]]
        outputDataDict = {}
        
        opframe = pd.DataFrame(columns=self.columns)
        coordDict = {}
    
        for ind in inputData.index:
            owner = inputData.loc[ind,OWNER]
            try:
                self.coordDict[owner]
                outputDataDict[owner]
            except KeyError:
                self.coordDict[owner]=[(inputData.loc[ind,LAT],inputData.loc[ind,LONG])]
                outputDataDict[owner]={ind:owner}
                opframe=opframe.append({
                    OWNER:owner,
                    ADDRESS:inputData[ADDRESS][ind],
                    OPACRES:inputData[ACRES][ind],
                    PARCELCOUNT:inputData[PARCELCOUNT][ind],
                    OPLAT:0,
                    OPLONG:0,
                    OPCLOSEST:0,
                    OPFARTHEST:0
                },ignore_index=True)
            self.coordDict[owner].append((inputData.loc[ind,LAT],inputData.loc[ind,LONG]))
        func = lambda x: x.replace(" ","%20")
        self.encodedAddress = list(map(func, opframe.loc[:,ADDRESS]))
        self.opframe = opframe
        if save:
            opframe.to_csv(self.outputDataPath,index=False)
#             return self.coordDict
        else:
            return self.opframe,self.coordDict
    
    def saveData(self,outputFilePath=None):
        if outputFilePath is None:
            self.opframe.to_csv(self.outputDataPath,index=False)
        else:
            self.opframe.to_csv(outputFilePath,index=False)
    
    def __getCoords__(self,URL):
        try:
            resp = req.get(URL)
            result = json.loads(resp.text)[0]
            print("Got result " + result)
            return (result['lat'],result['lon'])
        except KeyError:
            return getCoords(URL)
    
    def __requestCoords__(self,addresses):
        for address in addresses: 
            self.CoordFromAdd.append(self.__getCoords__(addressToCoordURL.format(apiKey,address)))
            
            
    def fillCoordFromAddress(self,numOfThreads=3,test=False):
        if test:
            addList = self.encodedAddress[:10]
        else:
            addList = self.encodedAddress
        threads = []
        batchSize = int(len(addList)/numOfThreads)
        for i in range(numOfThreads-1):
            th = Thread(target=self.__requestCoords__, args=(addList[i*batchSize:(i+1)*batchSize],))
            threads.append(th)
            print(i*batchSize,(i+1)*batchSize)
        threads.append(Thread(target=requestCoords, args=(addList[(numOfThreads-1)*batchSize:],)))
        print((numOfThreads-1)*batchSize,len(addList))
        for th in threads:
            th.start()
        for th in threads:
            th.join()
        self.CoordFromAdd = np.asarray(self.CoordFromAdd)
        if self.opframe is not None:
            if not test:
                self.opframe[:,self.OPLAT]=self.CoordFromAdd[:,0]
                self.opframe[:,self.OPLAT]=self.CoordFromAdd[:,1]
                return self.opframe
            else:
                temp = self.opframe.loc[:10]
                temp[:,self.OPLAT]=self.CoordFromAdd[:,0]
                temp[:,self.OPLAT]=self.CoordFromAdd[:,1]
                return temp
        
    def saveCoordToAdd(self,path):
        if self.CoordFromAdd is not None:
            cf = pd.DataFrame(columns=['address','lat','lon'])
            cf.loc[:,"address"]=self.opframe.loc[:,self.ADDRESS]
            cf.loc[:,"lat"]=self.CoordFromAdd[:,0]
            cf.loc[:,"lon"]=self.CoordFromAdd[:,1]
            cf.to_csv(path,index=False)
            print("Saved at " + path)
        else:
            print("CoordsFromAdd is None")
                   
        

In [ ]:
obj = processRec()
df,coord = obj.getUniqueData(False)

In [ ]:
frame = obj.fillCoordFromAddress(test=True)
print(frame.head())

In [ ]:
print(len(obj.CoordFromAdd))

In [ ]:
x = []
for ind in df.index:
    x.append([np.random.randint(100),np.random.randint(100)])
x = np.asarray(x)
print(x[:,1])

In [138]:
### new Project
import numpy as np
import PyPDF2 
import re
import pandas as pd

tupArr=[]
MODEL="Model"
UPC="UPC"
df = pd.DataFrame(columns=[MODEL,UPC])
filePath='garbage/UpworkProjects/utensils.pdf'

pdf = open(filePath,'rb')
pdfreader = PyPDF2.PdfFileReader(pdf)
numPages = pdfreader.getNumPages()

for pageNo in range(numPages):
    page = pdfreader.getPage(pageNo).extractText()
    arr=re.findall("Model:[\s]*[\n](.*?)[\s]*[\n][\s]*UPC:[\s]*(.*?)[\s]*[\n]",page)
    if len(arr) is not 0:#  and len(upc) is not 0
        tupArr.append(arr)
        pageswithDetails.append(pageNo)
        
for tup in tupArr:
    for model,upc in tup:
        df=df.append({MODEL:model,UPC:upc},ignore_index=True)
    
df.to_csv("garbage/UpworkProjects/Demo.csv",index=False)

[3, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16, 18, 20, 22, 23, 24, 25, 26, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 59, 60, 61, 62, 63]


In [185]:
from os import path
from glob import glob

dr='garbage/UpworkProjects/'
n="file.csv"
print(path.join(path.dirname(path.realpath(fileName)),outputFileName))
# d = "asdf_sdfas"
# print(bool(re.search(r"\s",d)))
# x = re.search('.pdf',dr)
# start,_=x.span()
# outputFile = dr[:start]+".csv"
# print(outputFile)
# i=0
# for filePath in glob(path.join(dr,"*.pdf")):
#     print(path.join(dr+"demo{}".format(i)+".csv"))
#     i+=1

C:\Users\Assessioner\ipynb files\garbage\file.csv
